# Evaluate Baselines

The goal of this notebook is to demonstrate how we can evaluate the results of a baseline on a given benchmark.

It will be split into two part. The first part will focus on the evaluation of a baseline that does not requires any training (the `DCApproximatrionAC`). On the second part, we will show how to load a baseline (or any other `AugmentedSimulator`) and evaluate it on a `Benchmark` of our choice.

As for the first notebook, we demonstrate this capability for the case of `NeuripsBenchmark1`.

**NB** This notebook supposes that the data for the benchmark are already available. If they are not, please generate them or download them.

**NB** The `DCApproximatrionAC` baseline requires the `grid2op` python package.

#### Import required packages

In [1]:
import pathlib
from pprint import pprint
from lips.benchmark.powergridBenchmark import PowerGridBenchmark
from lips.utils import get_path

In [2]:
# indicate required paths
LIPS_PATH = pathlib.Path().resolve().parent
DATA_PATH = LIPS_PATH / "reference_data" / "powergrid" / "l2rpn_case14_sandbox"
BENCH_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "benchmarks" / "l2rpn_case14_sandbox.ini"
SIM_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "simulators"
BASELINES_PATH = LIPS_PATH / "trained_baselines" / "powergrid"
EVALUATION_PATH = LIPS_PATH / "evaluation_results" / "PowerGrid"
LOG_PATH = LIPS_PATH / "lips_logs.log"

#  Benchmark1

## Initial step: load the dataset

A common dataset will be used for evaluate the two augmented simulator. This initial step aims at loading it once and for all.

In [3]:
benchmark1 = PowerGridBenchmark(benchmark_name="Benchmark1",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=BENCH_CONFIG_PATH
                               )

2022-05-14 18:33:09.767833: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [4]:
# to verify the config is loaded appropriately for this benchmark
print("Benchmark name: ", benchmark1.config.section_name)
print("Environment name: ", benchmark1.config.get_option("env_name"))
print("Output attributes: ", benchmark1.config.get_option("attr_y"))
print("Evaluation criteria: ")
pprint(benchmark1.config.get_option("eval_dict"))

Benchmark name:  Benchmark1
Environment name:  l2rpn_case14_sandbox
Output attributes:  ('a_or', 'a_ex')
Evaluation criteria: 
{'IndRed': [],
 'ML': ['MSE_avg', 'MAE_avg', 'mape_90_avg'],
 'OOD': ['MSE_avg', 'MAE_avg', 'mape_90_avg'],
 'Physics': ['CURRENT_POS']}


## The DC approximation

We remind that the `grid2op` library is required for this part. You can install it with `pip install grid2op` if you do not have it already.

First we will create the "augmented simulator". As opposed to the second model we will expose here, this method require access to a powergrid. This is one of the reason we need grid2op. 

The way to load each `AugmentedSimulator` is specific. Here for example we load the DCApproximation that will use the same powergrid as the one used to generate the data in the previous Notebook.

In [5]:
# the next few lines are specific for each benchmark and each `AugmentedSimulator`
import grid2op
import warnings
from lips.physical_simulator.dcApproximationAS import DCApproximationAS
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    env = grid2op.make(benchmark1.config.get_option("env_name"), test=True)
    grid_path = pathlib.Path(env.get_path_env()) / "grid.json"

dc_sim = DCApproximationAS(name="dc_approximation", 
                           benchmark_name="Benchmark1",
                           config_path=BENCH_CONFIG_PATH,
                           grid_path=grid_path)

Now that the model is load, there is a common interface to evaluate its performance, on a dataset. This is showed in the cell bellow where we evaluate a physics based simulator `DCApproximation` on these two dataset.

In [6]:
EVAL_SAVE_PATH = get_path(EVALUATION_PATH, benchmark1)
dc_metrics_per_dataset = benchmark1.evaluate_simulator(augmented_simulator=dc_sim,
                                                       dataset="all", # other values : "val", "test", "test_ood_topo"
                                                       save_path=EVAL_SAVE_PATH,
                                                       save_predictions=True
                                                      )

evaluate dc: 100%|██████████████████████████████████████████████████████████████████████████████████| 10000/10000 [06:03<00:00, 27.50it/s]


In [7]:
pprint(dc_metrics_per_dataset)

{'test': {'IndRed': {},
          'ML': {'MAE_avg': {'a_ex': 143.16579138645758,
                             'a_or': 114.3059446624192},
                 'MSE_avg': {'a_ex': 131732.91079220167,
                             'a_or': 91723.53006148033},
                 'mape_90_avg': {'a_ex': 0.18943869874874716,
                                 'a_or': 0.2054102762347278}},
          'Physics': {'CURRENT_POS': {}}},
 'test_ood_topo': {'IndRed': {},
                   'ML': {'MAE_avg': {'a_ex': 155.63737545351296,
                                      'a_or': 123.67567242785628},
                          'MSE_avg': {'a_ex': 155649.85692985042,
                                      'a_or': 107658.54513807358},
                          'mape_90_avg': {'a_ex': 0.20711185760666956,
                                          'a_or': 0.2227085624215881}},
                   'Physics': {'CURRENT_POS': {}}},
 'val': {'IndRed': {},
         'ML': {'MAE_avg': {'a_ex': 139.13596802494925,
       

And now it is possible to study the metrics on the different dataset. For example, if we want the "MSE" error on the "test" dataset (with a similar distribution as the training one):

## A learned baseline "augmented simulator"

Along with some dataset, we provide also some baseline (from a trained neural network). This baseline is made of a fully connected neural network that takes the available input of the powergrid and tries to predict all the output of the simulator.

The fully connected neural network is made of XXX layer each with YYY units.

It is learned for KKK epochs on the training set of the `Benchmark1`.

**NB** These baselines are not yet fully trained, and some hyper parameters still need to be optimized. We intend on doing that before the official release of the benchmark for the Neurips conference.

First we need to load the baseline and initialize it properly

In [ ]:
from lips.augmented_simulators.tensorflow_models import TfFullyConnected
from lips.dataset.scaler import StandardScaler

# rebuild the baseline architecture
tf_fc = TfFullyConnected(name="tf_fc",
                         bench_config_path=BENCH_CONFIG_PATH,
                         bench_config_name="Benchmark1",
                         sim_config_path=SIM_CONFIG_PATH / "tf_fc.ini",
                         sim_config_name="DEFAULT",
                         scaler=StandardScaler,
                         log_path=LOG_PATH)

LOAD_PATH = get_path(BASELINES_PATH, benchmark1)
tf_fc.restore(LOAD_PATH)

Then, as for the DC approximation, we can evaluate it on the test datasets of the benchmark.

This is done with the same command:

In [18]:
# TODO: log the losses
# EVAL_SAVE_PATH = get_path(EVALUATION_PATH, benchmark1)
tf_fc_metrics = benchmark1.evaluate_simulator(augmented_simulator=tf_fc,
                                              eval_batch_size=10000,
                                              dataset="all",
                                              shuffle=False,
                                              save_path=None,
                                              save_predictions=False
                                             )

2022-05-14 19:11:16.036827: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10


## Comparison of the two augmented simulator

### Machine learning metrics 

And now we can compare the two "augmented simulators". For example, if we want to compare the MAPE90 (mean absolute percentage error compute for last 10% quantile) on the test dataset (with a distribution similar to the training distribution) for currents (A) at two extremity of power lines, we might compare:

In [19]:
ML_metrics = "ML"
dataset_name = "test"
print("DC Approximation")
print(f"Dataset : {dataset_name}")
print("{:<10} : {}".format("MAPE90", dc_metrics_per_dataset[dataset_name][ML_metrics]["mape_90_avg"]))
print("{:<10} : {}".format("MSE_avg", dc_metrics_per_dataset[dataset_name][ML_metrics]["MSE_avg"]))
print("{:<10} : {}".format("MAE_avg", dc_metrics_per_dataset[dataset_name][ML_metrics]["MAE_avg"]))
dataset_name = "test_ood_topo"
print(f"Dataset : {dataset_name}")
print("{:<10} : {}".format("mape_90_avg", dc_metrics_per_dataset[dataset_name][ML_metrics]["mape_90_avg"]))
print("{:<10} : {}".format("MSE_avg", dc_metrics_per_dataset[dataset_name][ML_metrics]["MSE_avg"]))
print("{:<10} : {}".format("MAE_avg", dc_metrics_per_dataset[dataset_name][ML_metrics]["MAE_avg"]))

DC Approximation
Dataset : test
MAPE90     : {'a_or': 0.2054102762347278, 'a_ex': 0.18943869874874716}
MSE_avg    : {'a_or': 91723.53006148033, 'a_ex': 131732.91079220167}
MAE_avg    : {'a_or': 114.3059446624192, 'a_ex': 143.16579138645758}
Dataset : test_ood_topo
mape_90_avg : {'a_or': 0.2227085624215881, 'a_ex': 0.20711185760666956}
MSE_avg    : {'a_or': 107658.54513807358, 'a_ex': 155649.85692985042}
MAE_avg    : {'a_or': 123.67567242785628, 'a_ex': 155.63737545351296}


In [21]:
ML_metrics = "ML"
dataset_name = "test"
print("Fully Connected Augmented Simulator")
print(f"Dataset : {dataset_name}")
print("{:<10} : {}".format("mape_90_avg", tf_fc_metrics[dataset_name][ML_metrics]["mape_90_avg"]))
print("{:<10} : {}".format("MSE_avg", tf_fc_metrics[dataset_name][ML_metrics]["MSE_avg"]))
print("{:<10} : {}".format("MAE_avg", tf_fc_metrics[dataset_name][ML_metrics]["MAE_avg"]))
dataset_name = "test_ood_topo"
print(f"Dataset : {dataset_name}")
print("{:<10} : {}".format("mape_90_avg", tf_fc_metrics[dataset_name][ML_metrics]["mape_90_avg"]))
print("{:<10} : {}".format("MSE_avg", tf_fc_metrics[dataset_name][ML_metrics]["MSE_avg"]))
print("{:<10} : {}".format("MAE_avg", tf_fc_metrics[dataset_name][ML_metrics]["MAE_avg"]))

Fully Connected Augmented Simulator
Dataset : test
mape_90_avg : {'a_or': 0.005971284397153569, 'a_ex': 0.005959986222812791}
MSE_avg    : {'a_or': 21.65915870666504, 'a_ex': 44.23374557495117}
MAE_avg    : {'a_or': 2.7974419593811035, 'a_ex': 3.8893446922302246}
Dataset : test_ood_topo
mape_90_avg : {'a_or': 0.17957805279569528, 'a_ex': 0.17880120699511942}
MSE_avg    : {'a_or': 12364.6455078125, 'a_ex': 23608.12109375}
MAE_avg    : {'a_or': 52.66007614135742, 'a_ex': 74.55868530273438}


### Physic compliance

In [22]:
physic_compliances = "Physics"
dataset_name = "test"
current_violation = tf_fc_metrics[dataset_name][physic_compliances]["CURRENT_POS"]["a_or"]["Violation_proportion"]
print("{:.2f}% of currents at the origin side of power lines violate the current positivity.".format(current_violation*100))

2.23% of currents at the origin side of power lines violate the current positivity.


In [23]:
current_error = tf_fc_metrics[dataset_name][physic_compliances]["CURRENT_POS"]["a_or"]["Violation_proportion"]
print("The sum of negative current values (Amp) : {:.2f}".format(current_error))

The sum of negative current values (Amp) : 0.02


### Industrial readiness

In [24]:
tf_fc.predict_time

0.033661603927612305

In [25]:
dc_sim._predict_time

134.44738698005676

In [26]:
dc_sim._raw_grid_simulator.comp_time

76.7556830085814

# Benchmark2

# Benchmark3 